In [14]:
import streamlit as st

In [15]:
import pandas as pd

In [16]:
df=pd.read_csv('cleaned_Chicago_crime_3')

In [80]:
%%writefile streamlit_pro.py
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import plotly.graph_objects as go

st.set_page_config(layout='wide',
                  page_title = 'Chicago Crime')

st.title('Chicago Crime')
st.header('A 2001 - 2023 Analysis')


tab1, tab2 = st.tabs(['Over all Macros', 'Time series analysis'])
df=pd.read_csv('cleaned_Chicago_crime_3')
crime_count_of_1 = 1
num = df.describe()
cat = df.describe(include = 'O')
dataf= df.head(5)

with tab1:
    st.subheader('Chicago is world wide notorious for its reported high crime rate')
    st.subheader('This analysis is based on the data of actual Chicago police department cases gathered over a 22 year peiod from 2001 to 2023')
    col1 , col2 , col3 = st.columns(3)

    st.subheader('1- Most repeated crimes')
    crime_type = df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False)
    fig = px.bar(crime_type, x='Primary Type', y='Case Number', labels={'Primary Type': 'Crime Type', 'Case Number': 'Crime Count'})
    fig.update_layout(title_text='over all Crime rates by type')
    st.plotly_chart(fig,use_container_width=True)
        
    st.subheader('2- Geographic analysis of rimes')
    geocraphic_crime_pattern_top = df.groupby('new block address')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    fig = px.bar(geocraphic_crime_pattern_top, x='new block address', y='Case Number', labels={'new block address': 'Block Address', 'Case Number': 'Crime Count'})
    fig.update_layout(title_text='Geographic Crime Pattern by Block Address')
    st.plotly_chart(fig,use_container_width=True)
    
    district_crime_count_top = df.groupby('District')['Case Number'].count().reset_index().sort_values(by ='Case Number', ascending=False).head(5)
    fig = px.bar(district_crime_count_top, x='District', y='Case Number', labels={'District': 'District', 'Case Number': 'Crime Count'})
    fig.update_layout(title_text='Geographic Crime Pattern by District')
    st.plotly_chart(fig,use_container_width=True)

    geographic_crime_pattern_bottom = df.groupby('new block address')['Case Number'].count().reset_index()
    filtered_df = geographic_crime_pattern_bottom[geographic_crime_pattern_bottom['Case Number'] == crime_count_of_1]
    geographic_crime_pattern_bottom = filtered_df.sort_values(by='Case Number', ascending=False).head(20) 
    st.dataframe( geographic_crime_pattern_bottom)
        
    fig = px.bar(df['Location Description'].value_counts().head(10), title = '10 Most Dangerous places')
    st.plotly_chart(fig,use_container_width=True)
    
    fig = px.bar(df['Location Description'].value_counts().tail(20), title = '20 Least Dangerous places')
    st.plotly_chart(fig,use_container_width=True)

    st.subheader('3- Arrets analysis')
    
    arrest_count = df['Arrest'].value_counts()   
    fig = px.pie(arrest_count, values=arrest_count.values, names=arrest_count.index, title='Crimes with Arrests vs. No Arrests')
    st.plotly_chart(fig,use_container_width=True)
    
    filtered_df = df[df['Arrest'] == True]
    arrests_count_of_blocks_top = filtered_df.groupby('new block address')['Arrest'].count().reset_index().sort_values(by='Arrest', ascending = False)
    crimes_vs_arrest_in_top_locations = pd.merge(geocraphic_crime_pattern_top, arrests_count_of_blocks_top, on='new block address')
    crimes_vs_arrest_in_top_locations['arrest_rate'] = crimes_vs_arrest_in_top_locations['Arrest'] / crimes_vs_arrest_in_top_locations['Case Number'] * 100 
    new_pd = crimes_vs_arrest_in_top_locations.head(10) 
    new_pd2 = crimes_vs_arrest_in_top_locations.tail(10) 
    fig = px.line(new_pd, x='new block address', y=['Case Number', 'Arrest'], labels={'new block address': 'Block Address', 'Case Number': 'Count'}, title='Comparison of Case Numbers and Arrests by Block Address top 10')
    st.plotly_chart(fig,use_container_width=True)

    fig = px.bar(new_pd , x='new block address', y='arrest_rate', labels={'new block address': 'Block Address', 'arrest_rate': 'Arrest ratio'}, title= 'Top 10 Addresse for arrest rates')
    st.plotly_chart(fig,use_container_width=True)
    arrests_count_of_top_crimes = filtered_df.groupby('Primary Type')['Arrest'].count().reset_index().sort_values(by='Arrest', ascending = False)
    crime_type = df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False)
    crimes_type_vs_arrests = pd.merge(crime_type, arrests_count_of_top_crimes, on='Primary Type')
    new_df3 = crimes_type_vs_arrests.head(20)
    fig = px.line(new_df3, x='Primary Type', y=['Case Number', 'Arrest'], labels={'Primary Type': 'Crime Type', 'Case Number': 'Count'}, title='Comparison of Case Numbers and Arrests by top 10 Crime Types')
    st.plotly_chart(fig,use_container_width=True)
    new_df4 = crimes_type_vs_arrests.tail(20)
    fig = px.line(new_df4, x='Primary Type', y=['Case Number', 'Arrest'], labels={'Primary Type': 'Crime Type', 'Case Number': 'Count'}, title='Comparison of Case Numbers and Arrests by bottom 20 Crime Types')
    st.plotly_chart(fig,use_container_width=True)

    crimes_type_vs_arrests['arrest_rate'] = crimes_type_vs_arrests['Arrest'] / crimes_type_vs_arrests['Case Number'] * 100
    crimes_type_vs_arrests_top = crimes_type_vs_arrests.head(15)
    fig = px.bar(crimes_type_vs_arrests_top, x='Primary Type', y='arrest_rate', labels={'Primary Type': 'Crime type', 'arrest_rate': 'Arrest Ratio'}, title = 'Arrest rates of the top 10 repeated crimes')
    st.plotly_chart(fig,use_container_width=True)
    crimes_type_vs_arrests_bottom = crimes_type_vs_arrests.tail(20)
    fig = px.bar(crimes_type_vs_arrests_bottom, x='Primary Type', y='arrest_rate', labels={'Primary Type': 'Crime type', 'arrest_rate': 'Arrest Ratio'}, title = 'Arrest rates of the 20 least repeated crimes')
    st.plotly_chart(fig,use_container_width=True)
   
    st.subheader('4- Domestic Vs. Non Domestic Crimes')
                 
    Domistic_violence_count = df['Domestic'].value_counts()
    Domistic_violence = df['Domestic'].sum()
    fig = px.pie(Domistic_violence_count, values=Domistic_violence_count.values, names=Domistic_violence_count.index, title='Domestic vs. Non Domestic Crimes')
    st.plotly_chart(fig,use_container_width=True)

    filtered_df2 = df[(df['Domestic'] == True) & (df['Arrest'] == True)]
    Domestic_violence = df[df['Domestic'] == True].shape[0]  
    Arrests_of_domestic_crimes = filtered_df2.groupby('Domestic')['Arrest'].count().reset_index().sort_values(by='Arrest', ascending = False)
    Arrests_ratio_of_domestic_crimes = Arrests_of_domestic_crimes['Arrest'] / Domistic_violence *100
    arrests_ratio = len(filtered_df2) / Domestic_violence * 100 
    fig = go.Figure(
    data=go.Bar(
        x=['Arrest Ratio On Domestic Violence Crimes'],
        y=[arrests_ratio],
        text=[f'{arrests_ratio:.2f}%'],
        textposition='auto'
    ),
    layout=go.Layout(
        title='Arrest ratio on Domestic violence crimes'
    )
)
    st.plotly_chart(fig,use_container_width=True)


    col1 , col2 , col3 = st.columns(3)
    with col1:
        
        st.subheader('numerical descriptive statistics')
        st.dataframe(num)
    
    with col3:
        st.subheader('categorical descriptive statistics')
        st.dataframe(cat)

with tab2:
    
    st.subheader('1- Crime rates over time')
    crime_count_per_year = df.groupby('Year')['Case Number'].count().reset_index()
    fig = px.bar(crime_count_per_year, x='Year', y='Case Number', labels={'Year': 'Year', 'Case Number': 'Crime Count'}, title= 'Crime volume over 22 years')
    st.plotly_chart(fig,use_container_width=True)
    year = 2001
    filtered_df = df[df['Year'] == year]
    crime_type_count_2001 = filtered_df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    fig = px.bar(crime_type_count_2001, x='Primary Type', y='Case Number', labels={'Primary Type': 'Crime Type', 'Case Number': 'Crime Count'}, title = 'Top 5 crimes of 2001')
    st.plotly_chart(fig,use_container_width=True)
    year = 2012
    filtered_df = df[df['Year'] == year]
    crime_type_count_2012 = filtered_df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    fig = px.bar(crime_type_count_2012, x='Primary Type', y='Case Number', labels={'Primary Type': 'Crime Type', 'Case Number': 'Crime Count'}, title = 'Top 5 crimes of 2012')
    st.plotly_chart(fig,use_container_width=True)
    year = 2023
    filtered_df = df[df['Year'] == year]
    crime_type_count_2023 = filtered_df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    fig = px.bar(crime_type_count_2023, x='Primary Type', y='Case Number', labels={'Primary Type': 'Crime Type', 'Case Number': 'Crime Count'}, title = 'Top 5 crimes of 2023')
    st.plotly_chart(fig,use_container_width=True)
    

    st.subheader('2- Covid-19 effect on crime pattern')
    cyber_crime_count_2019 = filtered_df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    year = 2020
    filtered_df = df[df['Year'] == year]
    crime_type_count_2020 = filtered_df.groupby('Primary Type')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending=False).head(5)
    fig = px.bar(crime_type_count_2020, x='Primary Type', y='Case Number', labels={'Primary Type': 'Crime Type', 'Case Number': 'Crime Count'}, title= 'top 5 crimes during covid')
    st.plotly_chart(fig,use_container_width=True)

    cyber_crime = 'DECEPTIVE PRACTICE'
    filtered_df = df[df['Primary Type'] == cyber_crime]
    cyber_crimes_count = filtered_df.groupby('Year')['Case Number'].count().reset_index()   
    fig = px.line(cyber_crimes_count, x='Year', y='Case Number', labels={'Year': 'Year', 'Case Number': 'Cyber Crime'}, title= 'Cyber crimes over time')
    st.plotly_chart(fig,use_container_width=True)
    theft_crime = 'THEFT'
    filtered_df = df[df['Primary Type'] == theft_crime]
    theft_count = filtered_df.groupby('Year')['Case Number'].count().reset_index()
    fig = px.line(theft_count, x='Year', y='Case Number', labels={'Year': 'Year', 'Case Number': 'Theft'}, title = 'theft crimes over time')
    st.plotly_chart(fig,use_container_width=True)

    st.subheader('Note that cyber crime count went down by only 4% percent from 2019 to 2020 vs. other crimes such as theft that went down almost 34% for the same period. According to natiol statistics cyber crimes rate increased during the time of covid-19')
    
    
    st.subheader('3- Arrest rates over time')
    
    filterred_df = df[df['Arrest'] == True]
    arrests_over_time = filterred_df.groupby('Year')['Arrest'].count().reset_index().sort_values(by='Year', ascending =False)
    fig = px.line(arrests_over_time, x= 'Year', y='Arrest', title = 'Arrest rates over 22 years')
    st.plotly_chart(fig,use_container_width=True)
    crime_count_over_time = df.groupby('Year')['Case Number'].count().reset_index().sort_values(by='Year', ascending =False)
    merged_df = pd.merge(arrests_over_time, crime_count_over_time, on='Year')
    fig = px.line(merged_df, x='Year', y=['Case Number', 'Arrest'], labels={'Year': 'year', 'value': 'Count'}, title='Comparison of Case Numbers and Arrests over time')
    st.plotly_chart(fig,use_container_width=True)
    merged_df['Arrest_rates']= merged_df['Arrest'] / merged_df['Case Number'] * 100
    fig = px.line(merged_df, x= 'Year', y='Arrest_rates', title = 'Arrest rates over time')
    st.plotly_chart(fig,use_container_width=True)
    
    st.subheader('4- Average time for completion')
    
    avg_time_for_closing_cases = df.groupby('Year')['Date Difference'].median().reset_index().sort_values(by='Year', ascending =False)
    fig = px.line(avg_time_for_closing_cases, x= 'Year', y='Date Difference').update_layout(title_text='Average completion time')
    st.plotly_chart(fig,use_container_width=True)
    
    st.subheader('4- Seasonality')
    
    month_with_top_crime = df.groupby('Month')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending =True)
    fig = px.bar(month_with_top_crime, x='Case Number', y='Month', labels={'Case Number': 'Month', 'Month': ''}).update_layout(title_text='Crime Pattern by Month')
    st.plotly_chart(fig,use_container_width=True)
    fig = px.line(month_with_top_crime, x= 'Month', y='Case Number').update_layout(title_text='Crime Pattern by Month')
    st.plotly_chart(fig,use_container_width=True)
    filterred_df = df[df['Primary Type'] == 'HOMICIDE']
    Homicide_Monthly_pattern = filterred_df.groupby('Month')['Case Number'].count().reset_index().sort_values(by='Month', ascending =False)   
    #fig = px.bar(Homicide_Monthly_pattern, x='Month', y='Case Number', labels={'Case Number': 'Month', 'Case Number': 'Crime count'}).update_layout(title_text='Monthly Violent Crime Pattern')
    #st.plotly_chart(fig,use_container_width=True)
    fig = px.line(Homicide_Monthly_pattern, x= 'Month', y='Case Number').update_layout(title_text='Monthly Violent Crime Pattern')
    st.plotly_chart(fig,use_container_width=True)
    
    st.subheader('Time of Day analysis')
    
    
    time_with_top_crime = df.groupby('time')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending =False)
    time_with_top_5_crime= time_with_top_crime.head(5)
    fig = px.bar(time_with_top_5_crime, x='time', y='Case Number', labels={'Case Number': 'time', 'Case Number': 'Crime count'}).update_layout(title_text='Crime Pattern by Time of day')
    st.plotly_chart(fig,use_container_width=True)
    filterred_df = df[df['Primary Type'] == 'HOMICIDE']
    Homicide_day_time_pattern = filterred_df.groupby('time')['Case Number'].count().reset_index().sort_values(by='Case Number', ascending =False)
    Homicide_day_time_pattern_top5 =Homicide_day_time_pattern.head(5)
    fig = px.bar( Homicide_day_time_pattern_top5, x='time', y='Case Number', labels={'Case Number': 'time', 'Case Number': 'Crime count'}).update_layout(title_text='Violent Crime Pattern by Time of day')
    st.plotly_chart(fig,use_container_width=True)


Overwriting streamlit_pro.py


In [ ]:
! streamlit run streamlit_pro.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.3:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
2023-10-27 21:20:05.998 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert 'JC213529' with type str: tried to convert to int64", 'Conversion failed for column Case Number with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
